# Part 3 -- Get Target Data (S&P 500)

Get our target data (the entire S&P 500 index, made up of 500 individual stocks) and perform calculations. We want to look at the Close price to determine whether stocks went up/down/neutral compared to the previous day. This will be a **classification model** and we will set our threshold of up/down/neutral to a change of 2%.

**Load lib codes**

In [1]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/Analyzing_Unstructured_Data_for_Finance/')

from lib import *
# suppress_warnings()

In [2]:
!pip install pandas-datareader pymongo pyquery

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pymongo
import pandas_datareader.data as web
import bs4 as bs
import requests

**Web-scrape most recent list of S&P 500 stocks from Wikipedia and add to MongoDB**

In [4]:
# Identify port for better security of MongoDB
cli = pymongo.MongoClient(host='52.27.11.214', port=27016)

In [5]:
# Instantiates when you put data in 
sp500_collection = cli.stocks_db.sp500_collection
stocks_collection = cli.stocks_db.stocks_collection
cli.stocks_db.collection_names()

['sp500_collection', 'stocks_collection']

In [6]:
print(sp500_collection.count())
print(stocks_collection.count())

505
943687


In [106]:
# sp500_collection.drop()
# stocks_collection.drop()

In [109]:
def get_sp500_to_mongo():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'html5lib')
    table = soup.find('table', {'class': 'wikitable sortable'})
    for row in table.findAll('tr')[1:]:
        stock = row.findAll('td')[0].text
        sp500_dict = {
            'stock': stock,
            'status': 'pending'
        }
        sp500_collection.insert_one(sp500_dict)
get_sp500_to_mongo()

In [110]:
print(sp500_collection.count())
print(stocks_collection.count())

505
0


In [111]:
sp500_list = [document for document in sp500_collection.find()]
sp500_df = pd.DataFrame(sp500_list)
sp500_df.head()

,_id,status,stock
0,593e12ad57bbd405c4fb0f7f,pending,MMM
1,593e12ad57bbd405c4fb0f80,pending,ABT
2,593e12ad57bbd405c4fb0f81,pending,ABBV
3,593e12ad57bbd405c4fb0f82,pending,ACN
4,593e12ad57bbd405c4fb0f83,pending,ATVI


**Get stock data from Google Finance and add it to our sp500_collection in mongo**

In [112]:
# Takes argument of a stock and returns dataframe with stock column
def get_stocks(stock):
    try:
        start = dt.datetime(2009,6,11)
        end = dt.datetime(2017,6,11)
        df = web.DataReader(stock, 'google', start, end)
        df = df.reset_index()
        df['Stock'] = stock
        print('{} added for {}'.format(len(df), stock))
        return df
    except Exception as e:
        print('Error: e')
        pass

# Takes argument of a dataframe AND a mongo collection, turns it into a list of dictionaries and inserts into collection
def df_to_mongo(df, collection):
    try:
        stock_dict_list = list(df.to_dict(orient='index').values())
        collection.insert_many(stock_dict_list)
        sp500_collection.insert_one({'stock': stock, 'status': 'complete'})
        print('Successfully added to mongo')
    except Exception as e:
        sp500_collection.insert_one({'stock': stock, 'status': 'failed'})
        print('Error: e')
        pass

# Third function that does everything using both functions:
def get_stocks_to_mongo(stock):
    df = get_stocks(stock)
    df_to_mongo(df, stocks_collection)

In [113]:
start = datetime.now()

while True:
    # Get a user from mongo collection
    sp500 = sp500_collection.find_one_and_delete({'status': 'pending'})

    if type(sp500) != dict:
        print('Done!')
        break

    # Assign the stock to stock
    stock = sp500['stock']
        
    get_stocks_to_mongo(stock)

end = datetime.now()
print(end - start)

2014 added for MMM
Successfully added to mongo
2014 added for ABT
Successfully added to mongo
1133 added for ABBV
Successfully added to mongo
2014 added for ACN
Successfully added to mongo
2013 added for ATVI
Successfully added to mongo
2014 added for AYI
Successfully added to mongo
2013 added for ADBE
Successfully added to mongo
2012 added for AMD
Successfully added to mongo
2014 added for AAP
Successfully added to mongo
2014 added for AES
Successfully added to mongo
2014 added for AET
Successfully added to mongo
2014 added for AMG
Successfully added to mongo
2014 added for AFL
Successfully added to mongo
2014 added for A
Successfully added to mongo
2014 added for APD
Successfully added to mongo
2013 added for AKAM
Successfully added to mongo
2014 added for ALK
Successfully added to mongo
2014 added for ALB
Successfully added to mongo
2014 added for ARE
Successfully added to mongo
2013 added for ALXN
Successfully added to mongo
896 added for ALLE
Successfully added to mongo
930 added 

Successfully added to mongo
599 added for EQIX
Successfully added to mongo
2014 added for EQR
Successfully added to mongo
2014 added for ESS
Successfully added to mongo
2014 added for EL
Successfully added to mongo
2014 added for ES
Successfully added to mongo
2014 added for EXC
Successfully added to mongo
2013 added for EXPE
Successfully added to mongo
2013 added for EXPD
Successfully added to mongo
2013 added for ESRX
Successfully added to mongo
2014 added for EXR
Successfully added to mongo
2014 added for XOM
Successfully added to mongo
2013 added for FFIV
Successfully added to mongo
1274 added for FB
Successfully added to mongo
2013 added for FAST
Successfully added to mongo
2014 added for FRT
Successfully added to mongo
2014 added for FDX
Successfully added to mongo
2014 added for FIS
Successfully added to mongo
2013 added for FITB
Successfully added to mongo
2014 added for FE
Successfully added to mongo
2013 added for FISV
Successfully added to mongo
2013 added for FLIR
Successfu

2013 added for PCAR
Successfully added to mongo
2014 added for PH
Successfully added to mongo
2013 added for PDCO
Successfully added to mongo
2013 added for PAYX
Successfully added to mongo
488 added for PYPL
Successfully added to mongo
762 added for PNR
Successfully added to mongo
2013 added for PBCT
Successfully added to mongo
2014 added for PEP
Successfully added to mongo
2014 added for PKI
Successfully added to mongo
874 added for PRGO
Successfully added to mongo
2014 added for PFE
Successfully added to mongo
2014 added for PCG
Successfully added to mongo
2014 added for PM
Successfully added to mongo
1299 added for PSX
Successfully added to mongo
2014 added for PNW
Successfully added to mongo
2014 added for PXD
Successfully added to mongo
2014 added for PNC
Successfully added to mongo
2014 added for RL
Successfully added to mongo
2014 added for PPG
Successfully added to mongo
2014 added for PPL
Successfully added to mongo
2014 added for PX
Successfully added to mongo
2013 added for

In [7]:
print(sp500_collection.count())
print(stocks_collection.count())

505
943687


In [8]:
sp500_list = [document for document in sp500_collection.find()]
sp500_df = pd.DataFrame(sp500_list)
sp500_df.head()

,_id,status,stock
0,593e12b557bbd405c4fb1956,complete,MMM
1,593e12b657bbd405c4fb2135,complete,ABT
2,593e12b757bbd405c4fb25a3,complete,ABBV
3,593e12b857bbd405c4fb2d82,complete,ACN
4,593e12ba57bbd405c4fb3560,complete,ATVI


In [9]:
# Looks like we managed to pull all but 4 stocks
sp500_df[sp500_df['status']=='failed']

,_id,status,stock
284,593e13ec57bbd405c4033dab,failed,LMT
329,593e141a57bbd405c404748d,failed,NWL
338,593e142357bbd405c404a580,failed,NBL


In [10]:
# Getting ALL data out of a collection
stocks_list = [document for document in stocks_collection.find()]
stocks_df = pd.DataFrame(stocks_list)
stocks_df.sample(5)

,Close,Date,High,Low,Open,Stock,Volume,_id
26112,37.05,2012-09-04,37.48,36.71,36.94,A,3850002.0,593e12c357bbd405c4fb7785
875770,50.38,2010-07-29,51.58,50.06,51.46,VTR,1588412.0,593e14b357bbd405c4087047
239806,77.28,2011-02-24,79.24,76.51,78.91,COP,13165343.0,593e133957bbd405c4febab3
582215,8.04,2012-02-22,8.24,8.01,8.21,MU,19382392.0,593e140557bbd405c403f4f6
465169,35.27,2017-03-17,35.33,35.10,35.31,INTC,30778867.0,593e13bf57bbd405c4022b7f


In [12]:
stocks_df.shape

(943687, 8)

# NOTE: Can always change this from a classification model to regression model by using Percent_Change as our target instead of Percent_Change_Class

**We want the <u>PERCENT CHANGE</u> of each stock, so that data is normalized (this is especially important if we are going to make comparisons between stocks)**

In [13]:
def get_change_in_Close(df):
    list_of_df = []
    for s in df['Stock'].unique():
        try_df = df.loc[df['Stock']==s]
        try_df = try_df.sort(columns='Date')
        try_df['Diff'] = try_df['Close'].diff()
        try_df['Percent_Change'] = (try_df['Close']-try_df['Close'].shift(1))/try_df['Close']
        
        list_of_df.append(try_df)
        
    return list_of_df

In [14]:
start = datetime.now()

diff_list = get_change_in_Close(stocks_df)

end = datetime.now()
print(end - start)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """


0:00:53.998178


**We want to combine these individual stocks into one index (SP500)**<br>
To do this, we will take the average of the Percent_Change of all stocks for each day.

In [15]:
sp500_stocks_df = pd.DataFrame()
for i in range(len(diff_list)):
    try_df = pd.DataFrame(diff_list[i])
    sp500_stocks_df = sp500_stocks_df.append(try_df)

In [16]:
sp500_stocks_df

,Close,Date,High,Low,Open,Stock,Volume,_id,Diff,Percent_Change
0,60.58,2009-06-11,61.46,60.50,60.57,MMM,3627208.0,593e12b557bbd405c4fb1178,NaN,NaN
1,61.00,2009-06-12,61.00,59.91,60.37,MMM,3228967.0,593e12b557bbd405c4fb1179,0.42,0.006885
2,59.31,2009-06-15,60.40,58.71,60.40,MMM,4231551.0,593e12b557bbd405c4fb117a,-1.69,-0.028494
3,58.41,2009-06-16,59.57,58.25,59.41,MMM,3791473.0,593e12b557bbd405c4fb117b,-0.90,-0.015408
4,59.04,2009-06-17,59.44,58.59,58.59,MMM,3826144.0,593e12b557bbd405c4fb117c,0.63,0.010671
5,59.31,2009-06-18,59.95,58.84,58.89,MMM,3194453.0,593e12b557bbd405c4fb117d,0.27,0.004552
6,59.37,2009-06-19,60.19,58.93,59.48,MMM,5868096.0,593e12b557bbd405c4fb117e,0.06,0.001011
7,57.47,2009-06-22,59.17,57.42,58.62,MMM,5272874.0,593e12b557bbd405c4fb117f,-1.90,-0.033061
8,57.06,2009-06-23,57.74,56.61,57.74,MMM,4161156.0,593e12b557bbd405c4fb1180,-0.41,-0.007185
9,57.48,2009-06-24,58.55,57.16,57.46,MMM,4447603.0,593e12b557bbd405c4fb1181,0.42,0.007307


In [17]:
# Download data and fact-check results in Excel to see if calculations were done correctly
sp500_stocks_df.to_csv('../Analyzing_Unstructured_Data_for_Finance/data/3.2.sp500_stocks.csv')

In [18]:
sp500_stocks_df = sp500_stocks_df.pivot_table(columns='Date').T

In [20]:
sp500_stocks_df.shape

(2014, 7)

# NOTE: Might want to consider tweaking the threshold
(e.g. what percent change can my model detect?)

In [21]:
# Look at the distribution of Percent_Change to determine what threshold to use
print('min: ', sp500_stocks_df['Percent_Change'].min())
print('max: ', sp500_stocks_df['Percent_Change'].max())
print('mean: ', sp500_stocks_df['Percent_Change'].mean())

print('mode: ', "{0:.10f}".format(round(sp500_stocks_df['Percent_Change'],2).mode()[0]))
print('mode: ', "{0:.10f}".format(round(sp500_stocks_df['Percent_Change'],5).mode()[0]))

min:  -0.0815379205331
max:  0.051627944931
mean:  0.000329585358775
mode:  0.0000000000
mode:  -0.0011800000


In [28]:
# Make threshold .5% (for percent change in daily Close price)
def make_binary(data):
    data_list = []
    for d in data:
        if round(d,3) < 0.005:
            data_list.append('down')
        elif round(d,3) == 0.005:
            data_list.append('neutral')
        elif round(d,3) > 0.005:
            data_list.append('up')
        else:
            data_list.append(None)
    return data_list

In [29]:
sp500_stocks_df['Percent_Change_Class'] = make_binary(sp500_stocks_df['Percent_Change'])

In [30]:
sp500_stocks_df['Percent_Change_Class'].value_counts()

down       1391
up          536
neutral      86
Name: Percent_Change_Class, dtype: int64

In [31]:
sp500_stocks_df = sp500_stocks_df.reset_index()

In [32]:
sp500_stocks_df['Date'] = sp500_stocks_df['Date'].dt.date

In [34]:
sp500_stocks_df.shape

(2014, 9)

In [35]:
sp500_stocks_df.sample(5)

,Date,Close,Diff,High,Low,Open,Percent_Change,Volume,Percent_Change_Class
125,2009-12-08,38.845656,-0.314072,39.299774,38.451606,38.934887,-0.009200,7.592391e+06,down
1695,2016-03-07,81.530324,0.043907,82.207794,80.293198,81.016579,0.003840,5.020520e+06,down
604,2011-11-01,48.009207,-1.360815,48.961718,47.288436,47.880617,-0.030821,9.170421e+06,down
1170,2014-02-04,72.040127,0.673425,72.634989,71.027928,71.758858,0.008712,5.861444e+06,up
624,2011-11-30,49.053560,2.041626,49.292665,47.977885,48.388987,0.043303,9.073965e+06,up


In [36]:
pd.to_pickle(sp500_stocks_df, '../Analyzing_Unstructured_Data_for_Finance/data/3.2.sp500_stocks_df.pickle')